In [115]:
import openai, os, requests
from IPython.display import display, HTML, Markdown, JSON, Latex
import pandas as pd
import json


from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_type = "azure"
# Azure OpenAI on your own data is only supported by the 2023-08-01-preview API version
openai.api_version = "2023-08-01-preview"


# Azure OpenAI setup
openai.api_base = os.getenv("AZURE_OPEN_AI_ENDPOINT") # Add your endpoint here
openai.api_key = os.getenv("OPENAI_API_AZURE_KEY_REWILD") # Add your OpenAI API key here
deployment_id = "GPT4" # Add your deployment ID here

# Azure Cognitive Search setup
search_endpoint = os.getenv("AZURE_AI_SEARCH_ENDPOINT") # Add your Azure Cognitive Search endpoint here
search_key = os.getenv("AZURE_AI_SEARCH_RESOURCE_KEY_REWILD") # Add your Azure Cognitive Search admin key here
search_index_name = "plant-data-3-index" # Add your Azure Cognitive Search index name here


In [103]:
#plant_type choices tree, shrub, annual/perennial, others
#plant_size options = 0-1m, 1-5m, >5m
#flower_colour_input= red, orange, yellow, white, pink

In [116]:
prompt="""
Get me three plants that best match these requirements:\
plant_type:(plant_type_input) AND plant_size:(plant_size_input) AND flower_color:(flower_colour_input)\
If three options are not available for the requested criteria give me the closest matches.\
If multiple matches are found then prioritise the selected choices using the highest count value.\
Provide me the scientific name, common name and flower colour of the plants in a JSON object.\
Do not provide anything other than the JSON object.\

plant_type_input= shrub
plant_size_input= 1-5m
flower_colour_input= pink
"""

In [117]:
def setup_byod(deployment_id: str) -> None:
    """Sets up the OpenAI Python SDK to use your own data for the chat endpoint.
 
    :param deployment_id: The deployment ID for the model to use with your own data.
 
    To remove this configuration, simply set openai.requestssession to None.
    """
 
    class BringYourOwnDataAdapter(requests.adapters.HTTPAdapter):
 
        def send(self, request, **kwargs):
            request.url = f"{openai.api_base}/openai/deployments/{deployment_id}/extensions/chat/completions?api-version={openai.api_version}"
            return super().send(request, **kwargs)
 
    session = requests.Session()
 
    # Mount a custom adapter which will use the extensions endpoint for any call using the given `deployment_id`
    session.mount(
        prefix=f"{openai.api_base}/openai/deployments/{deployment_id}",
        adapter=BringYourOwnDataAdapter()
    )
    openai.requestssession = session
 




completion = openai.ChatCompletion.create(
    messages=[{"role": "user", "content": prompt,},
              {"role":"system", "content": "You are a native plant expert helping Australian suburban gardeners transform their gardens with local native plant recommendations. Use your knowledge base to provide recommendations and information"}
             ],
    deployment_id=deployment_id,
    dataSources=[  # camelCase is intentional, as this is the format the API expects
        {
            "type": "AzureCognitiveSearch",
            "parameters": {
                "endpoint": search_endpoint,
                "key": search_key,
                "indexName": search_index_name,
                "stream": "true",
                "stop": "null",
                "temperature": "0",
                "top_p": "1",
                "engine": "gpt-4",
                "topNDocuments": 20,
                "strictness": 2
                
            }
        }
    ]
)
print(completion)


{
  "id": "e7cf86bc-adac-4367-bc93-6317195caa02",
  "model": "gpt-4",
  "created": 1700945878,
  "object": "extensions.chat.completion",
  "choices": [
    {
      "index": 0,
      "finish_reason": "stop",
      "message": {
        "role": "assistant",
        "content": "The requested information is not available in the retrieved data. Please try another query or topic.",
        "end_turn": true,
        "context": {
          "messages": [
            {
              "role": "tool",
              "content": "{\"citations\": [{\"content\": \"https://upload.wikimedia.org/wikipedia/commons/thumb/9/9e/Angophora_hispida-Minchen.jpg/220px-Angophora_hispida-Minchen.jpg\\nNSW\\nAngophora hispida grows as a mallee, or as a tree to about 7 m (25 ft) in height.A. hispida's small size, especially when compared to its Angophora and Eucalyptus relatives, leads to it being known by the common name dwarf apple. It is native to a relatively small patch of central New South Wales \u2013 from just s

In [118]:
# Extract the text content from the 'completion' object
# Adjust the key access according to the actual structure of your 'completion' object
text_content = completion["choices"][0]["message"]["content"]

# Define the start and end markers for the "Description" section
# Ensure that these markers exactly match the text in the 'content'
start_marker = "```json\n"
end_marker = "\n```"

# Extract the section
start_index = text_content.find(start_marker)
end_index = text_content.find(end_marker)

if start_index != -1 and end_index != -1:
    # Adjust the start index to include the length of the start marker
    extracted_section = text_content[start_index + len(start_marker):end_index].strip()
else:
    extracted_section = "Section not found"


In [93]:
# Your JSON string
json_string = extracted_section

# Parse the JSON string
data = json.loads(json_string)

# Convert to DataFrame
df = pd.DataFrame(data)

# Display the DataFrame
df


,scientific_name,common_name,flower_color
0,Crowea saligna,Willow-leaved crowea,Mauve Pink
1,Elaeocarpus reticulatus,Blueberry ash,Pink
2,Ceratopetalum gummiferum,New South Wales Christmas bush,Pink Red White


In [119]:
prompt="""Provide me a summary of Crowea saligna expressed in a way that can be understood by a 12 year old./
The format should be as follows delimited in symbols and presented using markdown with the heading as\
described below. Use the information in the citation.
***
[Insert the common name here] (h2) 
Scientific name: (h3) 
Description: (h3) 
Requirements: (h3) 
Care: (h3)
Fun Factions: (h3)
^^^
"""

completion = openai.ChatCompletion.create(
    messages=[{"role": "user", "content": prompt }],
    deployment_id=deployment_id,
    dataSources=[  # camelCase is intentional, as this is the format the API expects
        {
            "type": "AzureCognitiveSearch",
            "parameters": {
                "endpoint": search_endpoint,
                "key": search_key,
                "indexName": search_index_name,
                "stream": "true",
                "stop": "null",
                "temperature": "0",
                "top_p": "1",
                "engine": "gpt-4",
            }
        }
    ]
)
print(completion)

{
  "id": "451bbf93-5847-45d5-a64d-66cf54d80ae9",
  "model": "gpt-4",
  "created": 1700945901,
  "object": "extensions.chat.completion",
  "choices": [
    {
      "index": 0,
      "finish_reason": "stop",
      "message": {
        "role": "assistant",
        "content": "***\n\n## Willow-leaved Crowea\n### Scientific name:\nCrowea saligna\n\n### Description:\nIt's a small bush that grows in eastern New South Wales, Australia. It has pretty pink flowers that look like stars and is often grown in gardens because it looks nice[doc1].\n\n### Requirements:\nThis plant likes places that aren't too bright, like moderate or dappled shade. It's a tough little shrub that grows slowly[doc1].\n\n### Care:\nWillow-leaved Crowea doesn't need too much attention, just a moderate amount of water and a bit of shade to keep it happy[doc1].\n\n### Fun Facts:\nIt's not just a pretty face; this plant is also pretty hardy, which means it can handle different weather without getting too fussy.\n\n^^^",
   

In [120]:
import json

# Extract the text content from the 'completion' object
# Adjust the key access according to the actual structure of your 'completion' object
text_content = completion["choices"][0]["message"]["content"]

# Define the start and end markers for the "Description" section
# Ensure that these markers exactly match the text in the 'content'
start_marker = "***"
end_marker = "^^^"

# Extract the section
start_index = text_content.find(start_marker)
end_index = text_content.find(end_marker)

if start_index != -1 and end_index != -1:
    # Adjust the start index to include the length of the start marker
    extracted_section = text_content[start_index + len(start_marker):end_index].strip()
else:
    extracted_section = "Section not found"

display(Markdown(extracted_section))


## Willow-leaved Crowea
### Scientific name:
Crowea saligna

### Description:
It's a small bush that grows in eastern New South Wales, Australia. It has pretty pink flowers that look like stars and is often grown in gardens because it looks nice[doc1].

### Requirements:
This plant likes places that aren't too bright, like moderate or dappled shade. It's a tough little shrub that grows slowly[doc1].

### Care:
Willow-leaved Crowea doesn't need too much attention, just a moderate amount of water and a bit of shade to keep it happy[doc1].

### Fun Facts:
It's not just a pretty face; this plant is also pretty hardy, which means it can handle different weather without getting too fussy.